In [ ]:
######## leg convention ########
# in : ket
# out : bra

# ttno.tensors["node_id"].shape = (neighbour_legs, out_legs, in_legs)
# cached_tensor.shape = (in_legs, neighbour_legs, out_legs)
# _contract_all_except_node = ( cache1_out, cache2_out, ttno_out, cache1_in , cache2_in, ttno_in)

# contracted_ttn , contraction_oder =  ttn2.completely_contract_tree(to_copy=True)
# contracted_ttn = [ contraction_oder[0]'s open_legs , contraction_oder[1]'s open_legs , .....]

In [2]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy

from pytreenet import contraction_util as cnt

In [3]:
from pytreenet.contractions.state_operator_contraction import contract_operator_tensor_ignoring_one_leg , contract_bra_tensor_ignore_one_leg

In [36]:
def random_big_ttns_two_root_children2():
    """
    Provides a ttns of the form
                0
               /
              /  
             1    
            / \\   
           /   \\    
          2     3    
               / \\
              /   \\
             4     5

    """
    shapes = [(7,2),(7,4,5,3),(4,4),(5,2,3,5),
                  (2,6),(3,7)]
    nodes = [ptn.random_tensor_node(shape, identifier="site"+str(i))
             for i, shape in enumerate(shapes)]
    random_ttns = ptn.TreeTensorNetworkState()
    random_ttns.add_root(nodes[0][0], nodes[0][1])
    random_ttns.add_child_to_parent(nodes[1][0],nodes[1][1],0,"site0",0)
    random_ttns.add_child_to_parent(nodes[2][0],nodes[2][1],0,"site1",1)
    random_ttns.add_child_to_parent(nodes[3][0],nodes[3][1],0,"site1",2)
    random_ttns.add_child_to_parent(nodes[4][0],nodes[4][1],0,"site3",1)
    random_ttns.add_child_to_parent(nodes[5][0],nodes[5][1],0,"site3",2)
    return random_ttns

ttn = random_big_ttns_two_root_children2()

matrix = ptn.random_hermitian_matrix((2*3*4*5*6*7))
matrix = matrix.reshape(2,3,4,5,6,7,2,3,4,5,6,7)
leg_dict = {"site"+str(i): i for i in range(6)}
ttno = ptn.TTNO.from_tensor(ttn, matrix, leg_dict)

MemoryError: Unable to allocate 388. MiB for an array with shape (5040, 5040) and data type complex128

In [33]:
for i in range(len(ttn.nodes)):
    print(find_tensor_leg_permutation(ttn,f"site{i}"))

(3, 0, 2, 1)
(3, 5, 7, 0, 2, 4, 6, 1)
(3, 0, 2, 1)
(3, 5, 7, 0, 2, 4, 6, 1)
(3, 0, 2, 1)
(3, 0, 2, 1)


In [34]:
for i in range(len(ttn.nodes)):
    print(ttn.tensors[f"site{i}"].shape)

(7, 2)
(7, 4, 5, 3)
(4, 4)
(5, 2, 3, 5)
(2, 6)
(3, 7)


In [ ]:
for i in range(len(ttn.nodes)):
    print(ttn.tensors[f"site{i}"].shape)
    
ttn.canonical_form("site4")
for i in range(len(ttn.nodes)):
    print(ttn.tensors[f"site{i}"].shape)    

In [30]:
def find_tensor_leg_permutation(ttn: ptn.TreeTensorNetwork, node_id: str) -> tuple[int,...]:
    """
        After contracting all the cached tensors to the site Hamiltonian, the
         legs of the resulting tensor are in the order of the Hamiltonian TTNO.
         However, they need to be permuted to match the legs of the site's
         state tensor. Such that the two can be easily contracted.
    """
    state_node = ttn.nodes[node_id]
    hamiltonian_node = ttn.nodes[node_id]
    permutation = []
    for neighbour_id in state_node.neighbouring_nodes():
        hamiltonian_index = hamiltonian_node.neighbour_index(neighbour_id)
        permutation.append(hamiltonian_index)
    output_legs = []
    input_legs = []
    for hamiltonian_index in permutation:
        output_legs.append(2*hamiltonian_index+3)
        input_legs.append(2*hamiltonian_index+2)
    output_legs.append(0)
    input_legs.append(1)
    output_legs.extend(input_legs)
    return tuple(output_legs)

In [59]:
tdvp = ptn.OneSiteTDVP(ttn, ttno, 0.1 , 10, {"site0": "A"})
print(tdvp.partial_tree_cache.keys())

tdvp.update_tree_cache("site4","site3")
tdvp.update_tree_cache("site5","site3")
tdvp.update_tree_cache("site3","site1")

dict_keys([('site0', 'site1'), ('site2', 'site1'), ('site1', 'site3'), ('site5', 'site3'), ('site3', 'site4')])


In [24]:
# initial partial tree cache is besed on (rev_update_path , next_node_id_dict[rev_update_path])
rev_update_path , next_node_id_dict = tdvp._find_caching_path()
rev_update_path , next_node_id_dict

(['site0', 'site2', 'site1', 'site5', 'site3', 'site4'],
 {'site0': 'site1',
  'site1': 'site3',
  'site3': 'site4',
  'site2': 'site1',
  'site5': 'site3'})

In [44]:
## update_tree_cache ## 

# ("site0", "site1")  --> contract_leaf
ket_node, ket_tensor = ttn["site0"]
bra_tensor = ket_tensor.conj()
ham_node, ham_tensor = ttno["site0"]
# ham_tensor = (4, 2, 2) , bra_tensor = (7, 2)
bra_ham = np.tensordot(ham_tensor, bra_tensor,
                       axes=(ham_node.nneighbours(),  # 1
                             ket_node.nneighbours())) # 1
bra_ham_ket = np.tensordot(ket_tensor, bra_ham,
                          axes=(ket_node.nneighbours(),  # 1 
                                ham_node.nneighbours())) # 1
# bra_ham_ket.shape = (7, 4 ,7)

# ("site1", "site3") ---> contract_subtrees_using_dictionary

ket_node, ket_tensor = ttn["site1"] # (7, 4, 5, 3)
tensor = cnt.contract_all_but_one_neighbour_block_to_ket(ket_tensor,
                                                         ket_node,
                                                         "site3",
                                                         tdvp.partial_tree_cache)
# tensor.shape = (5, 3, 4, 7, 16, 4)

tensor_ref = contract_operator_tensor_ignoring_one_leg(tensor,
                                                      ket_node,
                                                      ttno.tensors["site1"],
                                                      ttno.nodes["site1"],
                                                     "site3")

# _contract_all_except_node

In [4]:
tensor_ref = tdvp._contract_all_except_node("site4") 

tensor = ttno.tensors["site4"] # (36,6,6)
# for neighbour = site3:
cached_tensor = tdvp.partial_tree_cache.get_entry("site3","site4") # (2,36,2)
tensor = np.tensordot(tensor, cached_tensor, axes=((0,1))) # (6, 6, 2, 2)

axes = tdvp._find_tensor_leg_permutation("site4") # (3, 0, 2, 1)
tensor = np.transpose(tensor, axes=axes) # (6, 2, 6, 2)

np.allclose(tensor, tensor_ref)

True

In [5]:

tensor_ref = tdvp._contract_all_except_node("site3") 

tensor = ttno.tensors["site3"] # (576, 36, 49, 5, 5)
# for neighbour = site1 / site4/ site5:
cached_tensor = tdvp.partial_tree_cache.get_entry("site1","site3") # (5, 576, 5)
tensor = np.tensordot(tensor, cached_tensor, axes=((0,1))) # (36, 49, 5, 5, 5, 5)

cached_tensor = tdvp.partial_tree_cache.get_entry("site4","site3") # (2, 36, 2)
tensor = np.tensordot(tensor, cached_tensor, axes=((0,1))) # (49, 5, 5, 5, 5, 2, 2)

cached_tensor = tdvp.partial_tree_cache.get_entry("site5","site3") # (3, 49, 3)
tensor = np.tensordot(tensor, cached_tensor, axes=((0,1))) # (5, 5, 5, 5, 2, 2, 3, 3)

axes = tdvp._find_tensor_leg_permutation("site3") # (3, 5, 7, 0, 2, 4, 6, 1)
tensor = np.transpose(tensor, axes=axes) # (5, 2, 3, 5, 5, 2, 3, 5)

np.allclose(tensor, tensor_ref)

True

# _get_effective_site_hamiltonian

In [125]:
tensor = ptn.tensor_matricisation_half(tensor)

# _get_effective_link_hamiltonian

In [310]:
tdvp._split_updated_site("site3","site4")

ref_tensor = tdvp._get_effective_link_hamiltonian("site3","site4")

new_cache_tensor = tdvp.partial_tree_cache.get_entry("site3","site4")
other_cache_tensor = tdvp.partial_tree_cache.get_entry("site4","site3")
tensor = np.tensordot(new_cache_tensor,other_cache_tensor,axes=(1,1))
tensor = np.transpose(tensor, axes=[1,3,0,2]) 
tensor = ptn.tensor_matricisation_half(tensor)

np.allclose(tensor, ref_tensor)

tdvp.state.contract_nodes("link_site3_with_site4", "site4","site4")

# contract_neighbour_block_to_ket

In [315]:
tensor = cnt.contract_neighbour_block_to_ket(ket_tensor = tdvp.state.tensors["site1"],
                                             ket_node = tdvp.state.nodes["site1"],
                                             neighbour_id = "site2",
                                             partial_tree_cache = tdvp.partial_tree_cache,
                                            )
print(tdvp.state.tensors["site1"].shape)
print(tdvp.partial_tree_cache.get_entry("site2","site1").shape)
tensor.shape

(7, 5, 4, 3)
(4, 16, 4)


(7, 5, 3, 16, 4)

# contract_all_but_one_neighbour_block_to_ket

In [329]:
tensor = cnt.contract_all_but_one_neighbour_block_to_ket(ket_tensor = tdvp.state.tensors["site1"],
                                                         ket_node = tdvp.state.nodes["site1"],
                                                         next_node_id = "site0",
                                                         partial_tree_cache = tdvp.partial_tree_cache,
                                                         )
print( tdvp.state.tensors["site1"].shape)
print( tdvp.partial_tree_cache.get_entry("site3","site1").shape)
print(tdvp.partial_tree_cache.get_entry("site2","site1").shape)
print(tensor.shape)

(7, 5, 4, 3)
(5, 576, 5)
(4, 16, 4)
(7, 3, 576, 5, 16, 4)


In [317]:
result_tensor = tdvp.state.tensors["site1"]
# ket_node.neighbouring_nodes() = ['site0', 'site3', 'site2']
# for ket_node.neighbouring_nodes() != next_node_id
neighbour_id = "site3"
result_tensor = cnt.contract_neighbour_block_to_ket_ignore_one_leg(result_tensor,
                                                                   tdvp.state.nodes["site1"],
                                                                   neighbour_id,
                                                                   'site0',
                                                                    tdvp.partial_tree_cache)
neighbour_id = "site2"
result_tensor = cnt.contract_neighbour_block_to_ket_ignore_one_leg(result_tensor,
                                                                   tdvp.state.nodes["site1"],
                                                                   neighbour_id,
                                                                   'site0',
                                                                    tdvp.partial_tree_cache)

# contract_all_neighbour_blocks_to_ket

In [17]:
tensor = cnt.contract_all_neighbour_blocks_to_ket(ket_tensor = tdvp.state.tensors["site1"],
                                                  ket_node = tdvp.state.nodes["site1"],
                                                   partial_tree_cache = tdvp.partial_tree_cache)
print(tensor.shape)

(3, 4, 7, 576, 5, 16, 4)


# contract_neighbour_block_to_hamiltonian


In [332]:
tensor = cnt.contract_neighbour_block_to_hamiltonian(tdvp.hamiltonian.tensors["site3"],
                                                     tdvp.hamiltonian.nodes["site3"],
                                                     "site4",
                                                     tdvp.partial_tree_cache)
print(tdvp.hamiltonian.tensors["site3"].shape)
print(tdvp.partial_tree_cache.get_entry("site4","site3").shape)
print(tensor.shape)

(576, 36, 49, 5, 5)
(2, 36, 2)
(576, 49, 5, 5, 2, 2)


# contract_neighbour_block_to_hamiltonian_ignore_one_leg

In [9]:
tensor = cnt.contract_all_but_one_neighbour_block_to_hamiltonian(tdvp.hamiltonian.tensors["site3"],
                                                                 tdvp.hamiltonian.nodes["site3"],
                                                                 "site5",
                                                                 tdvp.partial_tree_cache)
print(tdvp.hamiltonian.tensors["site3"].shape)
print(tdvp.partial_tree_cache.get_entry("site1","site3").shape)
print(tdvp.partial_tree_cache.get_entry("site4","site3").shape)
print(tensor.shape)

(576, 36, 49, 5, 5)
(5, 576, 5)
(2, 36, 2)
(49, 5, 5, 5, 5, 2, 2)


# contract_all_neighbour_blocks_to_hamiltonian
# vs 
# _contract_all_except_node

In [21]:
tensor = cnt.contract_all_neighbour_blocks_to_hamiltonian(tdvp.hamiltonian.tensors["site3"],
                                                          tdvp.hamiltonian.nodes["site3"],
                                                          tdvp.partial_tree_cache)
print(tensor.shape)
# (ttno_out ,ttno_in, cache1_in, cache2_out, cache1_in, cache2_out, ... )
tensor2 = tdvp._contract_all_except_node("site3")
print(tensor2.shape)
# ( cache1_out, cache2_out, ttno_out, cache1_in,cache2, ttno_in)

(5, 5, 5, 5, 2, 2, 3, 3)
(5, 2, 3, 5, 5, 2, 3, 5)


# completely_contract_tree

In [5]:
ttn1 = deepcopy(ttn)

ttn1_ref, order = ttn.completely_contract_tree(to_copy=True)
print(order) 
print(ttn1_ref.shape)

ttn1.contract_nodes("site3","site4","site3")
ttn1.contract_nodes("site3","site5","site3")
ttn1.contract_nodes("site1","site2","site1")
ttn1.contract_nodes("site1","site3","site1")
ttn1.contract_nodes("site0","site1","site0")

print(ttn1.nodes["site0"].shape)
print(np.allclose(ttn1_ref, ttn1.tensors["site0"]))

['site0', 'site1', 'site2', 'site3', 'site4', 'site5']
(2, 3, 4, 5, 6, 7)
(2, 3, 4, 5, 6, 7)
True


# contract_two_ttns

In [120]:
from pytreenet.contractions.state_state_contraction import contract_any
# this contract_any is different from the one in the state_operator_contraction module
from pytreenet.contractions.state_state_contraction import contract_bra_to_ket_and_blocks 

In [177]:
ttn1 = ptn.random_big_ttns_two_root_children(mode = ptn.RandomTTNSMode.DIFFVIRT)
ttn2 = ptn.random_big_ttns_two_root_children(mode = ptn.RandomTTNSMode.DIFFVIRT)

In [154]:
dictionary = ptn.PartialTreeCachDict()
computation_order = ttn1.linearise()
print(computation_order)
for node_id in computation_order[:-1]: # The last one is the root node
        node = ttn1.nodes[node_id]
        parent_id = node.parent
        # Due to the linearisation the children should already be contracted.
        block = contract_any(node_id, parent_id,
                             ttn1, ttn2,
                             dictionary)
        dictionary.add_entry(node_id,parent_id,block)
        # The children contraction results are not needed anymore.
        children = node.children
        for child_id in children:
            dictionary.delete_entry(child_id,node_id)
print(dictionary.keys())
print(dictionary[('site1', 'site0')].shape)
print(dictionary[('site6', 'site0')].shape)

# contract_node_with_environment(ttn1.tensors["site0"],ttn1, ttn2,dictionary)

ketblock_tensor = cnt.contract_all_neighbour_blocks_to_ket(ttn1.tensors["site0"],
                                                           ttn1.nodes["site0"],
                                                           dictionary)
print(ketblock_tensor.shape)

result = contract_bra_to_ket_and_blocks(bra_tensor = ttn2.tensors["site0"], 
                                        ketblock_tensor = ketblock_tensor,
                                        bra_node = ttn2.nodes["site0"],
                                        ket_node = ttn1.nodes["site0"])
print(result)
print(ptn.contract_two_ttns(ttn1, ttn2))

['site2', 'site4', 'site5', 'site3', 'site1', 'site7', 'site6', 'site0']
dict_keys([('site1', 'site0'), ('site6', 'site0')])
(7, 7)
(6, 6)
(2, 7, 6)
(-276052.337676387-54403.522628234205j)
(-276052.337676387-54403.522628234205j)


In [199]:
ttn = ptn.random_big_ttns_two_root_children(mode = ptn.RandomTTNSMode.DIFFVIRT)
ttn.canonical_form("site0")
print(ptn.contract_two_ttns(ttn, ttn.conjugate()))
print(complex(np.tensordot(ttn.tensors["site0"],ttn.tensors["site0"].conj(),axes=((0,1,2),(0,1,2)))))

(1173361.3812407819+0j)
(1173361.3812407828+0j)
